In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm
import keras
from keras.losses import huber_loss
from keras.initializers import he_uniform

from keras import backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("train.csv")

In [3]:
dtest = pd.read_csv("test.csv")

In [4]:
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data =  data.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [6]:
data.head()

,y,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,88.53,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,76.26,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.shape

(4209, 369)

In [8]:
dtest

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
dtest =  dtest.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [10]:
dtest.head() 

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
dtest.shape

(4209, 368)

In [12]:
X_train, y_train = data.values[0:3499,1:], data.values[0:3499, :1].ravel()
X_valid, y_valid = data.values[3500:4208,1:], data.values[3500:4208, :1].ravel()
X_test = dtest.values[:, :]

In [13]:
print("X_train----",X_train.shape)
print("y_train----",y_train.shape)      
print("X_valid----",X_valid.shape)
print("y_valid----",y_valid.shape)
print("X_test-----",X_test.shape)

X_train---- (3499, 368)
y_train---- (3499,)
X_valid---- (708, 368)
y_valid---- (708,)
X_test----- (4209, 368)


In [14]:
X_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0.

In [15]:
y_train[0]

130.81

In [16]:
# y_train = min_max_scaler.fit_transform(y_train)


In [17]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [19]:
subsequences = 16
timesteps = X_train.shape[1]//subsequences

In [20]:
timesteps 

23

In [21]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_valid.reshape((X_valid.shape[0], subsequences, 1, timesteps, 1))
X_test  = X_test.reshape((X_test.shape[0], subsequences, 1, timesteps, 1))

In [22]:
X_train.shape

(3499, 16, 1, 23, 1)

In [23]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10)

In [24]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', recurrent_activation="hard_sigmoid",kernel_initializer='he_uniform',return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(64))
modelConvLSTM.add(Dense(32))
modelConvLSTM.add(Dense(1,kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='linear'))
modelConvLSTM.compile(optimizer='adam', loss="huber_loss")
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=700, verbose=2, callbacks=[early_stop], validation_data = (X_valid, y_valid))

Train on 3499 samples, validate on 708 samples
Epoch 1/700
 - 13s - loss: 92.7179 - val_loss: 45.6354
Epoch 2/700
 - 11s - loss: 29.4447 - val_loss: 19.6420
Epoch 3/700
 - 12s - loss: 15.7567 - val_loss: 11.5610
Epoch 4/700
 - 13s - loss: 11.7628 - val_loss: 9.3763
Epoch 5/700
 - 12s - loss: 9.6527 - val_loss: 8.0428
Epoch 6/700
 - 21s - loss: 8.3876 - val_loss: 7.3277
Epoch 7/700
 - 25s - loss: 7.6471 - val_loss: 7.3369
Epoch 8/700
 - 27s - loss: 7.0641 - val_loss: 6.1366
Epoch 9/700
 - 26s - loss: 6.5912 - val_loss: 6.3515
Epoch 10/700
 - 25s - loss: 6.3176 - val_loss: 5.6200
Epoch 11/700
 - 25s - loss: 6.1475 - val_loss: 5.9669
Epoch 12/700
 - 24s - loss: 5.9953 - val_loss: 5.2342
Epoch 13/700
 - 24s - loss: 5.9359 - val_loss: 5.1018
Epoch 14/700
 - 24s - loss: 5.7242 - val_loss: 5.0038
Epoch 15/700
 - 24s - loss: 5.5855 - val_loss: 5.4919
Epoch 16/700
 - 24s - loss: 5.5537 - val_loss: 5.3241
Epoch 17/700
 - 24s - loss: 5.5664 - val_loss: 5.6057
Epoch 18/700
 - 24s - loss: 5.7314 - 

In [25]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

{'Explained Variance Score': 0.6588569744009685,
 'Max Error': 38.006360168457036,
 'Mean Absolute Error': 5.213589379693156,
 'Mean Squared Error': 47.574056109418,
 'Root Mean Squared Error': 6.897394878460853,
 'Median Absolute Error': 4.4760952758789045,
 'R² Score': 0.6388068105848851,
 'Mean Absolute Percentage Error': 5.157290045920068}

In [26]:
ConvLSTMpred = modelConvLSTM.predict(X_test, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))


In [27]:

ConvLSTMpred 


array([ 77.061554,  95.72135 ,  77.383705, ...,  94.64857 , 113.89796 ,
        94.97267 ], dtype=float32)